# Basketball Playoffs Qualification

## Task description

Basketball tournaments are usually split in two parts. First, all teams play each other aiming to achieve the greatest number of wins possible. Then, at the end of the first part of the season, a pre determined number of teams which were able to win the most games are qualified to the playoff season, where they play series of knock-out matches for the trophy.

For the 10 years, data from players, teams, coaches, games and several other metrics were gathered and arranged on this dataset. The goal is to use this data to predict which teams will qualify for the playoffs in the next season.

## Data preparation

### Creating the database

First, we need to convert the CSV files to tables in an SQLite database, so we can analyze, manipulate and prepare data more easily. This was done with a couple of SQlite3 commands:

```
.mode csv
.import dataset/awards_players.csv awards_players
.import dataset/coaches.csv coaches
.import dataset/players.csv players
.import dataset/players_teams.csv players_teams
.import dataset/series_post.csv series_post
.import dataset/teams_post.csv teams_post
.import dataset/teams.csv teams
.save database.db
```

### Filtering unneeded rows and columns

Upon closer inspection of the dataset, we found some rows which had no effect or could have a negative impact in our models training, such as rows in the players table which corresponded to current coaches, and thus had no information related to their height, weight, etc.

## Model performance measures

### The Game Score measure
The Game Score measure, created by John Hollinger, attempts to give an estimation of a player's productivity for a single game. We will start working on our model based on this measure, applying it to each player based on a whole season's stats and dividing it by the amount of games played.

### Basketball Power Index


Import necessary packages

In [2]:
import sqlite3
import pandas as pd

Create dataframes based on the database and relations between data

In [10]:
con = sqlite3.connect("database.db")

# Player <-> Awards
pl_aw = pd.read_sql_query("SELECT * FROM awards_players INNER JOIN players ON awards_players.playerID = players.bioID", con)

# Player <-> Teams
pl_tm = pd.read_sql_query("SELECT * FROM players_teams INNER JOIN players ON players_teams.playerID = players.bioID", con)

# Teams <-> Post Season Results (aggregated)
tm_psa = pd.read_sql_query("SELECT * FROM teams_post INNER JOIN teams ON (teams_post.tmID = teams.tmID AND teams_post.year = teams.year)", con)

# Coach <-> Teams
cc_tm = pd.read_sql_query("SELECT * FROM coaches INNER JOIN teams ON (coaches.tmID = teams.tmID AND coaches.year = teams.year)", con)


,coachID,year,tmID,lgID,stint,won,lost,post_wins,post_losses,year,...,GP,homeW,homeL,awayW,awayL,confW,confL,min,attend,arena
0,adamsmi01w,5,WAS,WNBA,0,17,17,1,2,5,...,34,11,6,6,11,9,11,6850,214448,Verizon Center
1,adubari99w,1,NYL,WNBA,0,20,12,4,3,1,...,32,12,4,8,8,14,7,6425,231962,Madison Square Garden (IV)
2,adubari99w,2,NYL,WNBA,0,21,11,3,3,2,...,32,13,3,8,8,13,8,6400,250565,Madison Square Garden (IV)
3,adubari99w,3,NYL,WNBA,0,18,14,4,4,3,...,32,10,6,8,8,11,10,6450,234717,Madison Square Garden (IV)
4,adubari99w,4,NYL,WNBA,0,16,18,0,0,4,...,34,11,6,5,12,11,13,6850,212346,Madison Square Garden (IV)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,wintebr01w,6,IND,WNBA,0,21,13,2,2,6,...,34,14,3,7,10,14,6,6925,142494,Conseco Fieldhouse
158,wintebr01w,7,IND,WNBA,0,21,13,0,2,7,...,34,12,5,9,8,12,8,6850,122768,Conseco Fieldhouse
159,wintebr01w,8,IND,WNBA,0,21,13,3,3,8,...,34,12,5,9,8,12,8,6875,133915,Conseco Fieldhouse
160,zierddo99w,8,MIN,WNBA,0,10,24,0,0,8,...,34,7,10,3,14,8,14,6875,121019,Target Center
